In [ ]:
import cv2
import numpy as np
import os
import imutils
import pandas as pd
from google.colab import drive
import numpy as np
drive.mount('/content/drive')
path = '/content/drive/MyDrive/CNN/photo mass detector/'
print("Исходный путь к рабочей директории: ", os.getcwd())
os.chdir(path)
print("Новый путь к рабочей директории: ", os.getcwd())
from google.colab.patches import cv2_imshow
from PIL import Image

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Исходный путь к рабочей директории:  /content
Новый путь к рабочей директории:  /content/drive/MyDrive/CNN/photo mass detector


In [ ]:
#отрисовка контуров на первичном сыром изображении
def image_contour(image):
  raw = image.copy()

  # Примените фильтр медианного размытия
  blurred_image = cv2.medianBlur(image, 5)

  # Создайте фильтр уточнения границ
  kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])

  # Примените фильтр к изображению
  sharp_image = cv2.filter2D(blurred_image, -1, kernel)

  # Преобразуйте изображение в градации серого
  gray = cv2.cvtColor(sharp_image, cv2.COLOR_BGR2GRAY)

  # Создайте экземпляр фильтра CLAHE
  clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

  # Примените фильтр CLAHE к градациям серого изображения
  contrast_enhanced = clahe.apply(gray)

  _, thresh = cv2.threshold(contrast_enhanced, 250, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)


  # Поиск контуров на бинаризованном изображении

  #contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  contours, _ = cv2.findContours(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

  return(contours, raw, contrast_enhanced)

#отрисовка контура чашки на изображении, чтобы потом по нему создать маску
def image_petri_cnt(raw, cnts_petri):

  #отрисовка незаметного контура
  raw_new = cv2.drawContours(raw, [cnts_petri], -1, (0, 1, 0), 2)
  #cv2_imshow(raw_new)
  # создаём маску по контуру чашки петри
  gray = cv2.cvtColor(raw, cv2.COLOR_RGB2GRAY)
  mask = np.zeros(gray.shape,np.uint8)
  mask = cv2.drawContours(mask, [cnts_petri], -1, 255, cv2.FILLED)
  fg_masked = cv2.bitwise_and(raw_new, raw_new, mask=mask)

  # белый фон (используем инвертированную маску)
  mask = cv2.bitwise_not(mask)
  bk = np.full(raw_new.shape, 255, dtype=np.uint8)
  bk_masked = cv2.bitwise_and(bk, bk, mask=mask)

  # сливаем фон и передний план
  final = cv2.bitwise_or(fg_masked, bk_masked)
  return(final)

#обрезает изображение по контуру чашки петри,после чего можно отрисовывать
#контуры внутри чашки(потенциально полезные)
#final - изображение, которое будет обрезаться.
#path_cutted_petri - путь, куда сохранится обрезанное изображение
def image_petri_cnt_cutter(final, path_cutted_petri):
  #код для разложения и обрезки изображения по маске контура круга чашки петри
  gray = cv2.cvtColor(final, cv2.COLOR_RGB2GRAY)
  thresh = cv2.threshold(gray, 195, 255, cv2.THRESH_BINARY)[1]
  thresh = cv2.bitwise_not(thresh)
  cnts_in_petri = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts_in_petri = imutils.grab_contours(cnts_in_petri)
  cnts_in_petri = sorted(cnts_in_petri, key=cv2.contourArea, reverse=True)[:10]

  #обработка каждого контура
  i = 0
  for c in cnts_in_petri:
    if cv2.contourArea(c) > 1000:

      #рисование маски, оставляем контур
      mask = np.zeros(gray.shape, np.uint8)
      mask = cv2.drawContours(mask, [c], -1, 255, cv2.FILLED)

      #белый фон
      fg_masked = cv2.bitwise_and(raw, raw, mask=mask)
      mask = cv2.bitwise_not(mask)
      bk = np.full(raw.shape, 255, dtype=np.uint8)
      bk_masked = cv2.bitwise_and(bk, bk, mask=mask)
      finalcont = cv2.bitwise_or(fg_masked, bk_masked)

      #ограничивающая область по контуру
      output = finalcont.copy()
      x,y,w,h = cv2.boundingRect(c)

      #вырезание области с чашкой
      roi = finalcont[y:y+h, x:x+w]

      #команда для изменения размера изображения если оно будет подаваться в нейросеть
      #roi = cv2.resize(roi, (400,400))

      #сохраняем картинку
      cv2.imwrite(path_cutted_petri, roi)
      #cv2_imshow(roi)
      i += 1
      return(roi)

#функция разархивации для датасетов
#archive_format = "zip"
def archive_extractor(path, extract_dir, archive_format):
  import shutil
  shutil.unpack_archive(path, extract_dir, archive_format)
  print("Archive file unpacked successfully.")

def filenames_finder(path):
  filenames = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
  return(filenames)


In [ ]:
path = '/content/drive/MyDrive/CNN/photo mass detector/regression model low train.zip'
extract_dir = '/content/drive/MyDrive/CNN/photo mass detector/regression model'
archive_format = "zip"
archive_extractor(path, extract_dir, archive_format)

ReadError: /content/drive/MyDrive/CNN/photo mass detector/regression model low train.zip is not a zip file

In [ ]:
#НАРЕЗКА СЫРЫХ ИЗОБРАЖЕНИЙ НА ROI
#путь с папкой сырых изображений
path_roi = '/content/drive/MyDrive/CNN/photo mass detector/fenton/test'

#путь куда сохранять обрезанные изображения
path_cutted_roi = '/content/drive/MyDrive/CNN/photo mass detector/fenton/roi'

#создание списка имен изображений в папке
filenames_Mn = filenames_finder(path_roi)

#удаление имени объекта .ipynb из списка с именами изображений
filenames_Mn.pop(-1)
filenames_Mn = sorted(filenames_Mn)

#цикл перебирает каждое исходное изображение, обрезает до чашки, сохраняет в новую папку
for el in filenames_Mn:
  print(el)
  path_cut = os.path.join(path_roi, el)
  test1 = cv2.imread(path_cut)
  contours, raw, contrast_enhanced = image_contour(test1)

  #путь сохранения обрезанного изображения чашки петри с растениями(roi)
  path_cutted_petri = os.path.join(path_cutted_roi, el)

  #сортировка контуров в изначальном изображении, чтобы найти контур чашки
  cnts_petri = sorted(contours, key=cv2.contourArea, reverse=True)[1]

  #удаление всей внешней области фотографии, не входящей в контур петри
  final = image_petri_cnt(raw, cnts_petri)

  #обрезка изображения по контуру чашки петри
  roi =  image_petri_cnt_cutter(final, path_cutted_petri)


001 1.JPG
001 2.JPG
001 3.JPG
001 4.JPG
003 1.JPG
003 2.JPG
003 4.JPG
control 1.JPG
control 2.JPG
control 3.JPG
